# Pre-processing - RD
Here I will pre-process the data for the R&D dataset of the LHC Olympics 2020. I will calculate some variables to use them on classification algorithms. Ultimately this will multiple functions in the benchtools package.

In [2]:
# Importing the main libraries
import h5py                             # to handle .h5 files
import pyjet as fj                      # jet clustering
import numpy as np                      # for arrays
import pandas as pd                     # manipulation of tables
import os.path                          # to handle directories
from os import path
from tqdm import tqdm                   # progress bar

In [ ]:
# We load the file with the data as a dataframe
# The file can be downloaded at https://zenodo.org/record/4536624
df = pd.read_hdf("../../events_anomalydetection.h5")

In [3]:
# Choose the number of events to analize
n_eventos = 100000

In [ ]:
# Random sample of the events
dfsample = df.sample(n=n_events)

# And save it on an .h5 file for reproducibility
if path.exists("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos))!= True: 
    dfsample.to_hdf("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos), key='df', mode='w',complevel=5,complib='blosc')

In [4]:
# We load it on a dataframe (if the file is already created there is no need to run the previous cell)
eventos_tiny = pd.read_hdf("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos))

In [5]:
# Verifying the shape (n_events x nro. hadrones*3(caracteristicas)+1(señal))
eventos_tiny.shape

(100000, 2101)

### Functions to calculate the variables

I will calculate the variables:
#### Angular distance between two jets

$$
\Delta R = \sqrt{(\phi_1-\phi_2)^2+(\eta_1-\eta_2)^2}
$$
  
#### N-Subjettiness

The 0-, 1- and 2-subjettiness are defined as:

$$
\begin{align}
\tau_0(\beta) &= \sum_{i\in J} p_{T_i}\Delta R^\beta \\
\tau_1(\beta) &= \frac{1}{\tau_0(\beta)} \sum_{i\in J} p_{T_i}\Delta R_{a_1,i}^\beta \\
\tau_2(\beta) &= \frac{1}{\tau_0(\beta)}\sum_{i\in J} p_{T_i} \text{min}(\Delta R_{a1,i}^\beta,\Delta R_{a_2,i})
\end{align}
$$
    To generate a dimensionless variable:
$$
\tau_{21}=\frac{\tau_2}{\tau_1}
$$

In [6]:
def deltaR(x, y):
    return ((x.phi-y.phi)**2 + (x.eta-y.eta)**2)**0.5

def subjettiness(cndts, cnsts):
    d0 = sum(c.pt for c in cnsts)
    ls = []
    for c in cnsts:
        dRs = [deltaR(c,cd) for cd in cndts]
        ls += [c.pt * min(dRs)]
    return sum(ls)/d0

def tau21(jet,subR=0.2):
    '''Input: jet from the jet clustering result '''
    jet_substruct_features = {}        
    seq = fj.cluster(jet, R=subR, algo='kt')
    cnsts = jet.constituents()
    cndts1 = seq.exclusive_jets(1)
    tau1 = subjettiness(cndts1, cnsts)
    if (len(cnsts)>1):
        cndts2 = seq.exclusive_jets(2)
        tau2 = subjettiness(cndts2, cnsts)
    else: 
        tau2 = 0
    
    try:
        return tau2/tau1
    
    except ZeroDivisionError:
        return 0

### Clustering and table
We will do the clustering of the data separately for the signal and for the background, so we will create a function that does the grouping and generates a table. The table will contain $p_T$, $m$, $\eta$, $\phi$, $E$, $\tau_{21}$, $m_{jj}$ and $\Delta R_{12}$ of the two main jets, as well as the * no. hadrons * of the event. 

In [7]:
def tabla(data):
    n_events = data.shape[0]                   # no. of events (1000)
    n_hadrones_gen = int((data.shape[1]-1)/3)  # no. of hadrons (700) 
                                               # [-1 to eliminate the label column, /3 for the 3 values for each hadron]
    data_ss = data.iloc[:,:-1]                 # the dataframe without labels
    
    # Defining the variables
    df = pd.DataFrame(columns=['pT_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 'tau_21_j1',  
                                'pT_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2', 'tau_21_j2',
                                'm_jj', 'deltaR_j12', 'label'])

    for event in tqdm(range(n_events)):

        pseudojets_input = np.zeros(len([data for data in data_ss.iloc[event,::3] if data > 0]), dtype= fj.DTYPE_PTEPM) 

        for hadron in range(n_hadrones_gen):
            if (data_ss.iloc[event,hadron*3] > 0): ## si pT > 0 

                ## Filling the array with pT, eta y phi for each hadron
                pseudojets_input[hadron]['pT'] = data_ss.iloc[event,hadron*3] 
                pseudojets_input[hadron]['eta'] = data_ss.iloc[event,hadron*3+1]
                pseudojets_input[hadron]['phi'] = data_ss.iloc[event,hadron*3+2]

                pass
            pass

        ## Returns a "ClusterSequence" (pyjet type of list)
        secuencia = fj.cluster(pseudojets_input, R=1.0, p=-1) 

        ## With inclusive_jets you get all the clustered jets
        ## and filter the ones with pT greater than 20.
        
        ## Making a list of the pseudojets
        jets = secuencia.inclusive_jets(ptmin=20) 

        # Getting the variables from the most energetic jet
        pT_j1 = jets[0].pt
        m_j1 = np.abs(jets[0].mass)
        eta_j1 = jets[0].eta
        phi_j1 = jets[0].phi
        E_j1 = jets[0].e
        tau_21_j1= tau21(jets[0])

        # Try getting the variables for the second most energetic jet (if it exists)
        try:
            pT_j2 = jets[1].pt
            m_j2 = np.abs(jets[1].mass)
            eta_j2 = jets[1].eta
            phi_j2 = jets[1].phi
            E_j2 = jets[1].e
            tau_21_j2= tau21(jets[1])
    
        # If not, all zero
        except IndexError:
            pT_j2 = 0
            m_j2 = 0
            eta_j2 = 0
            phi_j2 = 0
            E_j2 = 0
            tau_21_j2 = 0
        
        # Calculating the other variables
        deltaR_j12 = deltaR(jets[0], jets[1])
        mjj = m_j1 + m_j2
        n_hadrones = data.iloc[event,:].astype(bool).sum(axis=0)/3
        label = data.iloc[event,-1]

        # Adding it to the dataframe
        entry = pd.DataFrame([[pT_j1, m_j1, eta_j1, phi_j1, E_j1, tau_21_j1,  
                                pT_j2, m_j2, eta_j2, phi_j2, E_j2, tau_21_j2, 
                                mjj,deltaR_j12, n_hadrones, label]],
                            columns=['pT_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 'tau_21_j1',  
                                    'pT_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2', 'tau_21_j2',
                                    'm_jj', 'deltaR_j12', 'n_hadrones', 'label'])
        df = df.append(entry, sort=True)
    return df

Generating the data:

In [8]:
df = tabla(eventos_tiny)
df.head()

100%|██████████| 100000/100000 [55:44<00:00, 29.90it/s] 


,E_j1,E_j2,deltaR_j12,eta_j1,eta_j2,label,m_j1,m_j2,m_jj,n_hadrones,pT_j1,pT_j2,phi_j1,phi_j2,tau_21_j1,tau_21_j2
0,1541.103551,1314.832020,3.046675,-0.676879,-0.479236,0.0,140.326113,53.455092,193.781205,212.0,1239.698604,1176.086061,-0.949919,2.090338,0.672299,0.556964
0,1735.554053,1456.732975,3.175913,0.487036,0.215936,0.0,140.960881,91.964940,232.925821,160.0,1543.152632,1420.578669,0.580209,-2.584112,0.429885,0.709729
0,1747.277774,2131.876916,3.196916,-0.561281,-0.915332,0.0,130.198271,595.900967,726.099237,245.0,1499.892043,1412.636020,-2.168662,1.008589,0.682710,0.489799
0,1955.619525,2206.429386,3.483775,0.608165,-0.867853,0.0,388.869494,37.471626,426.341119,116.0,1609.609763,1574.869196,-0.565932,2.589707,0.740089,0.559426
0,1810.986292,2424.718009,3.642805,-0.664116,1.154562,0.0,242.616470,201.110818,443.727289,146.0,1460.557601,1385.597608,-1.685001,1.471332,0.437715,0.566019


Saving the dataframe as a csv:

In [9]:
outname = 'eventosRD_{}.csv'.format(n_eventos)
outdir = '../data'
# If the path does not exist, then create it
if not os.path.exists(outdir):
    os.makedirs(outdir)

path = os.path.join(outdir, outname)    
df.to_csv(path, sep=',', index=False)